<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/master/Initial_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the victim's gender using an evolutionary algorithm to select features.

* ## Initial Population

**ENTRADA:** População Inicial (aleatória)

*   Função de Fitness
*   Critério de Parada

**REPITA (até que o critério de parada seja atendido):**

*  PASSO 1: Aplicar a função de fitness a cada indivíduo

*  PASSO 2: Selecionar os x melhores indivíduos

*  PASSO 3: Reprodução
  * Aplicar o crossover a um par (com prob = p)
  * Aplicar mutação (com prob = p’)

* PASSO 4: Formar uma nova população com os filhos gerados

**SAÍDA:** Melhor indivíduo presente na geração final

##Imports

In [ ]:
!pip install category_encoders

In [28]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import read_csv 
from category_encoders import TargetEncoder

## Scripts

In [2]:
def database(url):
  df_encoded = pd.read_csv(url)
  df_encoded = df_encoded.drop(columns=['Unnamed: 0'])

  return df_encoded

In [3]:
def population(ind_size,pop_size,models):
  ind = []
  pop = []

  for p in range(pop_size):
    ind = [random.choice([True,False]) for i in range(ind_size)]
    ind[0] = random.choice(range(0,len(models)+1))
    pop.append(ind)
    ind = []

  return pop

In [12]:
#Modelo 
def model(population,df,models):
  result = []

  x = df[df.columns.difference(['Victim Sex'])]
  y = df[['Victim Sex']]
  
  x_train, x_test, y_train, y_test = np.array(train_test_split(x, y, test_size=0.2, stratify=y, random_state=42))

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.ravel(y_train)
  y_test = np.array(y_test)

  for p in population:
    model = models[p[1]]
    ind = [i==1 for i in p[1:]] 
    model.fit(x_train[:,ind],y_train)
    y_pred = model.predict(x_test[:,ind])
    accuracy = accuracy_score(y_test,y_pred)

    result.append([p,accuracy])

  return result

## Instances

In [5]:
df_encoded = database('/content/drive/My Drive/Data Files/df_model_victim_encoded')

In [6]:
models = [DecisionTreeClassifier(max_depth=10),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=14),RandomForestClassifier(max_depth=10)]

In [13]:
initial_population = population(18,100,models)

In [21]:
initial_model = model(initial_population,df_encoded,models)

In [29]:
#export 
with open("/content/drive/My Drive/Data Files/initial_model.txt", "wb") as fp: 
  pickle.dump(initial_model, fp)